In [41]:
%load_ext autoreload
%autoreload 2

import os 
import sys
sys.path.append("../")

from vector_graph.bipartite_graph_dict import BipartiteGraphDict
from utils.parse_arxiv import make_keyword_id, parse_authors, parse_categories
import pandas as pd
import pdb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
chunks = pd.read_json(
    "../data/arxiv-metadata-oai-snapshot.json", lines=True, chunksize=50000
)

keyword_author_ids = set()



keyword_category_ids = set()
data_ids = set()
edges = []

for chunk in chunks:
    df = chunk[["id", "authors", "categories"]]
    for i, row in df.iterrows():
        data_id = str(row["id"])
        # print(parse_authors(row["authors"])[0])
        # print(data_id)
        # print(parse_categories(row['categories']))
        keyword_author_id = make_keyword_id("authors", parse_authors(row["authors"])[0])
        # keyword_category_id = make_keyword_id("categories", parse_categories(row['categories']))
        edges.append((data_id, keyword_author_id))

        # edges.append((data_id, keyword_category_id))

        keyword_author_ids.add(keyword_author_id)

        data_ids.add(data_id)

        # keyword_category_ids.add(keyword_category_id)

In [43]:
G = BipartiteGraphDict()
G.add_data_nodes(data_ids)
G.add_keyword_nodes(keyword_author_ids)
G.add_raw_edges(edges)

In [76]:
# Perform a Simple Test
print("Num data points: ", len(data_ids))
print("Num Authors: ", len(keyword_author_ids))

query_list = G.get_data_ids_sorted_by_num_edges()[:4]
print(query_list)
most_relevant_res = G.find_relevant(
    query_list,
    100,
)
print(len(most_relevant_res))
# add metadata to most_relevant res results so that can see keywords in common with query, keywords and nodes traversed to find result
print(most_relevant_res)
print(G.get_common_keywords_from_id_list(most_relevant_res))

Num data points:  2358543
Num Authors:  990452
['2206.02196', 'math/9803042', 'q-alg/9602017', '1208.5337']
num rounds 2
35
['1203.6769', '1307.8309', '1207.0687', '1203.1747', '1205.6088', '1208.0453', 'math/9807119', '708.2391', 'math/0404499', 'math/9807116', 'math/0606093', 'math/0403188', '805.4', 'math/9807118', 'math/9807117', 'math/0412413', 'math/9804072', 'math/9807115', 'math/0006065', 'math/9807120', 'math/0405087', 'math/9802066', 'math/0102215', 'math/0506578', 'math/0401423', 'math/9807114', 'math/0105233', 'math/0307345', '1911.01203', '2103.04327', '2109.05024', '2005.10346', '2006.01601', '2011.02342', '2011.04079']
[('authors:ArturoMagidin', 22), ('authors:AlexanderJ.M.Kell', 7), ('authors:MajidHamzavi', 6)]
